<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Parts-of-Speech-Tagging-Begginer/blob/main/NLP_Parts_of_Speech_Tagging_Begginer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import numpy as np
import pandas as pd

In [3]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos

--2022-04-15 12:00:45--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_24.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286063 (279K) [text/plain]
Saving to: ‘WSJ_24.pos.1’

WSJ_24.pos.1        100%[===================>] 279.36K  --.-KB/s    in 0.02s   

2022-04-15 12:00:45 (12.4 MB/s) - ‘WSJ_24.pos.1’ saved [286063/286063]



In [4]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos

--2022-04-15 12:00:46--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/WSJ_02-21.pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8279089 (7.9M) [text/plain]
Saving to: ‘WSJ_02-21.pos.1’

WSJ_02-21.pos.1     100%[===================>]   7.90M  --.-KB/s    in 0.07s   

2022-04-15 12:00:46 (115 MB/s) - ‘WSJ_02-21.pos.1’ saved [8279089/8279089]



In [5]:
!wget https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt

--2022-04-15 12:03:04--  https://raw.githubusercontent.com/amanjeetsahu/Natural-Language-Processing-Specialization/master/Natural%20Language%20Processing%20with%20Probabilistic%20Models/Week%202/hmm_vocab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196571 (192K) [text/plain]
Saving to: ‘hmm_vocab.txt’

hmm_vocab.txt       100%[===================>] 191.96K  --.-KB/s    in 0.02s   

2022-04-15 12:03:04 (10.4 MB/s) - ‘hmm_vocab.txt’ saved [196571/196571]



In [6]:
with open('WSJ_02-21.pos', 'r') as f:
  train_corpus = f.readlines()

In [8]:
train_corpus[10: 20]

['at\tIN\n',
 'Chicago\tNNP\n',
 "'s\tPOS\n",
 'Goodman\tNNP\n',
 'Theatre\tNNP\n',
 '(\t(\n',
 '``\t``\n',
 'Revitalized\tVBN\n',
 'Classics\tNNS\n',
 'Take\tVBP\n']

In [9]:
with open('WSJ_24.pos', 'r') as f:
  y = f.readlines()

In [10]:
y[10: 20]

['points\tNNS\n',
 'this\tDT\n',
 'week\tNN\n',
 ',\t,\n',
 'with\tIN\n',
 'readings\tNNS\n',
 'on\tIN\n',
 'trade\tNN\n',
 ',\t,\n',
 'output\tNN\n']

In [20]:
with open('hmm_vocab.txt', 'r') as f:
  vocabulary_list = f.read().split('\n')

vocab = {}
for ind, word in enumerate(sorted(vocabulary_list)):
  vocab[word] = ind

In [21]:
sorted(vocabulary_list)[10: 20]

["'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m"]

In [22]:
vocab

{'': 0,
 '!': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 "''": 7,
 "'40s": 8,
 "'60s": 9,
 "'70s": 10,
 "'80s": 11,
 "'86": 12,
 "'90s": 13,
 "'N": 14,
 "'S": 15,
 "'d": 16,
 "'em": 17,
 "'ll": 18,
 "'m": 19,
 "'n'": 20,
 "'re": 21,
 "'s": 22,
 "'til": 23,
 "'ve": 24,
 '(': 25,
 ')': 26,
 ',': 27,
 '-': 28,
 '--': 29,
 '--n--': 30,
 '--unk--': 31,
 '--unk_adj--': 32,
 '--unk_adv--': 33,
 '--unk_digit--': 34,
 '--unk_noun--': 35,
 '--unk_punct--': 36,
 '--unk_upper--': 37,
 '--unk_verb--': 38,
 '.': 39,
 '...': 40,
 '0.01': 41,
 '0.0108': 42,
 '0.02': 43,
 '0.03': 44,
 '0.05': 45,
 '0.1': 46,
 '0.10': 47,
 '0.12': 48,
 '0.13': 49,
 '0.15': 50,
 '0.17': 51,
 '0.19': 52,
 '0.2': 53,
 '0.24': 54,
 '0.25': 55,
 '0.3': 56,
 '0.32': 57,
 '0.375': 58,
 '0.4': 59,
 '0.43': 60,
 '0.45': 61,
 '0.5': 62,
 '0.53': 63,
 '0.59': 64,
 '0.6': 65,
 '0.60': 66,
 '0.7': 67,
 '0.71': 68,
 '0.75': 69,
 '0.8': 70,
 '0.88': 71,
 '0.9': 72,
 '0.94': 73,
 '0.95': 74,
 '1': 75,
 '1,000': 76,
 '1,012': 77,
